In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Define image dimensions and other parameters
image_height = 150
image_width = 150
batch_size = 32
epochs = 5
validation_split = 0.2


In [3]:
# Define the CNN architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))


/opt/anaconda3/envs/fyp/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=validation_split
)


In [5]:
# Load images from the 'Archive' folder with subdirectories 'real' and 'fake'
train_generator = datagen.flow_from_directory(
    'dataset',
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)


Found 6057 images belonging to 2 classes.


In [6]:
# Use the validation subset for validation data
val_generator = datagen.flow_from_directory(
    'dataset',
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


Found 1513 images belonging to 2 classes.


In [7]:
# Train the model
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)


Epoch 1/5


/opt/anaconda3/envs/fyp/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


190/190 ━━━━━━━━━━━━━━━━━━━━ 83s 424ms/step - accuracy: 0.8602 - loss: 0.3036 - val_accuracy: 0.9511 - val_loss: 0.1865
Epoch 2/5
190/190 ━━━━━━━━━━━━━━━━━━━━ 85s 435ms/step - accuracy: 0.9815 - loss: 0.0535 - val_accuracy: 0.9451 - val_loss: 0.2752
Epoch 3/5
190/190 ━━━━━━━━━━━━━━━━━━━━ 85s 438ms/step - accuracy: 0.9921 - loss: 0.0243 - val_accuracy: 0.9570 - val_loss: 0.4184
Epoch 4/5
190/190 ━━━━━━━━━━━━━━━━━━━━ 87s 450ms/step - accuracy: 0.9886 - loss: 0.0307 - val_accuracy: 0.9504 - val_loss: 0.3377
Epoch 5/5
190/190 ━━━━━━━━━━━━━━━━━━━━ 89s 456ms/step - accuracy: 0.9915 - loss: 0.0208 - val_accuracy: 0.9597 - val_loss: 0.4203


In [8]:
# Print training and validation accuracy
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
print(f'Training accuracy: {train_acc[-1]}')
print(f'Validation accuracy: {val_acc[-1]}')


Training accuracy: 0.9915800094604492
Validation accuracy: 0.9596827626228333


In [9]:
# Save the model
model.save('real_or_fake.h5')
